In [1]:
#Model development for Trawling4PACE
#Authors: HS, VA, FB, PP
#History: 1/29/26
#Goals: Run Random forest and boosted regression tree models for predicting silver hake abundance

In [52]:
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [5]:
os.chdir('/home/jovyan/2026-proj-Trawling4PACE/data')

In [61]:
df= pd.read_csv('/home/jovyan/2026-proj-Trawling4PACE/contributor_folders/leandro/correlation_data.csv')

In [54]:
df

,DECDEG_ENDLAT,DECDEG_ENDLON,BEGIN_GMT_TOWDATE,BOTTEMP,BOTSALIN,SURFSALIN,SURFTEMP,EXPCATCHNUM_log,EXPCATCHWT_log,curl_stress_curl,fsle_fsle_max,glorys_bottomT_surface,glorys_mlotst_surface,glorys_so_surface,glorys_thetao_surface
0,39.945217,-73.419478,03/22/2024 06:43:58,7.05,32.015,31.993,7.03,1.146128,1.338138,-1.909989e-07,-0.051181,6.978496,23.705259,31.827396,7.090406
1,39.945217,-73.419478,03/22/2024 06:43:58,7.05,32.015,31.993,7.03,2.247973,1.916628,-1.909989e-07,-0.051181,6.978496,23.705259,31.827396,7.090406
2,39.945217,-73.419478,03/22/2024 06:43:58,7.05,32.015,31.993,7.03,2.152288,0.246006,-1.909989e-07,-0.051181,6.978496,23.705259,31.827396,7.090406
3,39.945217,-73.419478,03/22/2024 06:43:58,7.05,32.015,31.993,7.03,1.278754,1.081887,-1.909989e-07,-0.051181,6.978496,23.705259,31.827396,7.090406
4,39.945217,-73.419478,03/22/2024 06:43:58,7.05,32.015,31.993,7.03,0.477121,-0.521434,-1.909989e-07,-0.051181,6.978496,23.705259,31.827396,7.090406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7465,34.933743,-75.329215,03/13/2024 10:30:51,15.34,36.009,35.937,20.51,0.000000,-0.586700,9.812857e-07,-0.201945,15.024970,36.012746,36.325038,21.829062
7466,34.933743,-75.329215,03/13/2024 10:30:51,15.34,36.009,35.937,20.51,0.698970,-0.105130,9.812857e-07,-0.201945,15.024970,36.012746,36.325038,21.829062
7467,34.933743,-75.329215,03/13/2024 10:30:51,15.34,36.009,35.937,20.51,0.000000,-0.692504,9.812857e-07,-0.201945,15.024970,36.012746,36.325038,21.829062
7469,34.933743,-75.329215,03/13/2024 10:30:51,15.34,36.009,35.937,20.51,0.301030,-1.142668,9.812857e-07,-0.201945,15.024970,36.012746,36.325038,21.829062


In [62]:
#Data cleaning 
#all values in glorys_thetao_bottom are NAN so we need to clean them out 
df = df.drop(['glorys_thetao_bottom','BEGIN_GMT_TOWDATE'],axis=1).dropna()

#PLACEHOLDER for standardizing variables / outlier removal 
scaler = MinMaxScaler()
df_scaled = scaler.fit_transform(df)
#X_test_scaled = scaler.fit_transform(X_test)
#y_train_scaled = scaler.fit_transform(y_train)
#y_test_scaled = scaler.fit_transform(y_test)


In [63]:
#SPLIT DATA INTO TRAINING/TESTING 
y = df['EXPCATCHNUM_log']
X = df[['DECDEG_ENDLAT', 'DECDEG_ENDLON','BOTTEMP',
       'BOTSALIN', 'SURFSALIN', 'SURFTEMP', 'curl_stress_curl', 'fsle_fsle_max',
       'glorys_bottomT_surface', 'glorys_mlotst_surface', 'glorys_so_surface',
       'glorys_thetao_surface']]

X_train_scaled, X_test_scaled, y_train_scaled, y_test_scaled = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
#hyperparameterization
param_grid={'n_estimators':[100,200,300],
            'max_features':['sqrt','log2',0.8],
           'max_depth':[10,20,30,None]}
from sklearn.model_selection import GridSearchCV
model=RandomForestRegressor()
grid_search = GridSearchCV(estimator=model,
                          param_grid=param_grid,
                          cv=5,
                          scoring='neg_mean_squared_error',
                          n_jobs=-1)

# Fit GridSearchCV to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters found
print(grid_search.best_params_)

# Print the best score achieved (neg_mean_squared_error, so lower is better for actual MSE)
print(grid_search.best_score_)

In [68]:
#n_estimators == # of decision trees 
#oob_score == out of bag estimate of models general error
#random_state is for reproducibility

#optimize the number of trees?
#hyperparameterization
regressor = RandomForestRegressor(n_estimators=1000, random_state=0, oob_score=True)

regressor.fit(X_train, y_train)

,"n_estimators n_estimators: int, default=100The number of trees in the forest... versionchanged:: 0.22 The default value of ``n_estimators`` changed from 10 to 100 in 0.22.",1000
,"criterion criterion: {""squared_error"", ""absolute_error"", ""friedman_mse"", ""poisson""}, default=""squared_error""The function to measure the quality of a split. Supported criteriaare ""squared_error"" for the mean squared error, which is equal tovariance reduction as feature selection criterion and minimizes the L2loss using the mean of each terminal node, ""friedman_mse"", which usesmean squared error with Friedman's improvement score for potentialsplits, ""absolute_error"" for the mean absolute error, which minimizesthe L1 loss using the median of each terminal node, and ""poisson"" whichuses reduction in Poisson deviance to find splits.Training using ""absolute_error"" is significantly slowerthan when using ""squared_error""... versionadded:: 0.18 Mean Absolute Error (MAE) criterion... versionadded:: 1.0 Poisson criterion.",'squared_error'
,"max_depth max_depth: int, default=NoneThe maximum depth of the tree. If None, then nodes are expanded untilall leaves are pure or until all leaves contain less thanmin_samples_split samples.",None
,"min_samples_split min_samples_split: int or float, default=2The minimum number of samples required to split an internal node:- If int, then consider `min_samples_split` as the minimum number.- If float, then `min_samples_split` is a fraction and `ceil(min_samples_split * n_samples)` are the minimum number of samples for each split... versionchanged:: 0.18 Added float values for fractions.",2
,"min_samples_leaf min_samples_leaf: int or float, default=1The minimum number of samples required to be at a leaf node.A split point at any depth will only be considered if it leaves atleast ``min_samples_leaf`` training samples in each of the left andright branches. This may have the effect of smoothing the model,especially in regression.- If int, then consider `min_samples_leaf` as the minimum number.- If float, then `min_samples_leaf` is a fraction and `ceil(min_samples_leaf * n_samples)` are the minimum number of samples for each node... versionchanged:: 0.18 Added float values for fractions.",1
,"min_weight_fraction_leaf min_weight_fraction_leaf: float, default=0.0The minimum weighted fraction of the sum total of weights (of allthe input samples) required to be at a leaf node. Samples haveequal weight when sample_weight is not provided.",0.0
,"max_features max_features: {""sqrt"", ""log2"", None}, int or float, default=1.0The number of features to consider when looking for the best split:- If int, then consider `max_features` features at each split.- If float, then `max_features` is a fraction and `max(1, int(max_features * n_features_in_))` features are considered at each split.- If ""sqrt"", then `max_features=sqrt(n_features)`.- If ""log2"", then `max_features=log2(n_features)`.- If None or 1.0, then `max_features=n_features`... note:: The default of 1.0 is equivalent to bagged trees and more randomness can be achieved by setting smaller values, e.g. 0.3... versionchanged:: 1.1 The default of `max_features` changed from `""auto""` to 1.0.Note: the search for a split does not stop until at least onevalid partition of the node samples is found, even if it requires toeffectively inspect more than ``max_features`` features.",1.0
,"max_leaf_nodes max_leaf_nodes: int, default=NoneGrow trees with ``max_leaf_nodes`` in best-first fashion.Best nodes are defined as relative reduction in impurity.If None then unlimited number of leaf nodes.",None
,"min_impurity_decrease min_impurity_decrease: float, default=0.0A node will be split if this split induces a decrease of the impuritygreater than or equal to this value.The weighted impurity decrease equation is the following:: N_t / N * (impurity - N_t_R / N_t * right_impurity - N_t_L / N_t * left_impurity)where ``N`` is the total number of samples, ``N_t`` is the number ofsampl

In [69]:
from sklearn.metrics import mean_squared_error, r2_score

oob_score = regressor.oob_score_
print(f'Out-of-Bag Score: {oob_score}')

predictions = regressor.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, predictions)
print(f'R-squared: {r2}')

Out-of-Bag Score: -0.06361059573788275
Mean Squared Error: 0.7496454738639804
R-squared: -0.02835322359334458


In [44]:
predictions

array([1.10434348, 0.73104434, 0.56237032, 0.73363292, 0.93556661,
       1.20646701, 0.69394763, 0.51768316, 0.30624624, 0.7741243 ,
       0.97675981, 0.76237019, 0.97020157, 0.73363292, 0.92152932,
       0.83035307, 0.84036507, 1.29672909, 0.76237019, 0.76550752,
       0.91387917, 0.67765275, 0.88112523, 0.88510306, 0.75319603,
       0.12014594, 0.92018628, 0.97675981, 1.05863133, 0.83015199,
       1.0745604 , 0.73104434, 0.73211603, 0.67234391, 0.95447033,
       1.1210474 , 0.83015199, 0.67234391, 0.96207574, 0.89172567,
       0.85333357, 0.98312413, 0.91935461, 0.67789908, 0.46629601,
       0.56089082, 1.1648721 , 0.62580667, 0.79550275, 0.46878365,
       1.14089881, 0.31218118, 0.90916299, 1.04102429, 1.1210474 ,
       0.55982457, 0.56916166, 0.72382857, 0.83536136, 0.75319603,
       0.99628247, 0.31218118, 0.95406285, 1.46763011, 1.0011376 ,
       0.79035414, 0.67765275, 0.31218118, 0.65264576, 0.59156898,
       0.94511156, 0.76550752, 1.59990566, 0.73363292, 0.93556